In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather

In [10]:
import glob
import os

In [66]:
#path for 11 models
path = "/home/mlopez/EXEC/Tg_annual_11_models/"

In [67]:
files = list(glob.glob(os.path.join(path,'*.*')))

In [68]:
files

['/home/mlopez/EXEC/Tg_annual_11_models/ACCESS1-3_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/BNU-ESM_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/CanESM2_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/CMCC-CMS_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/GFDL-ESM2M_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/HadGEM2-CC_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/INM-CM4_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/IPSL-CM5A-LR_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/MPI-ESM-LR_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/NorESM1-M_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/IPSL-CM5B-LR_rcp45_tg_mean_annual.nc']

# Create DF of all models

In [69]:
dflist = []

In [71]:
for file in files:
    df = pcdf.load_as_df(file)
    dflist.append(df)

/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between

In [72]:
dflist

[                                    tg_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  258.828888
                      1951-01-01  258.898285
                      1952-01-01  258.709839
                      1953-01-01  258.461304
                      1954-01-01  259.764862
 ...                                     ...
 40.041039 -74.045807 2096-01-01  287.267639
                      2097-01-01  287.343201
                      2098-01-01  289.033142
                      2099-01-01  287.839813
                      2100-01-01  287.281799
 
 [11507106 rows x 1 columns],                                     tg_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  257.955170
                      1951-01-01  256.791321
                      1952-01-01  259.168579
                      1953-01-01  258.776642
                      1954-01-01  257.527466
 ...                                     ...
 40.041039 -74.045807 20

In [73]:
#Takes 3 min
%time df11 = pd.concat(dflist)

CPU times: user 3min 17s, sys: 3min 38s, total: 6min 56s
Wall time: 59.9 s


In [74]:
df11

tg_mean
lat       lon        time                  
66.623306 -89.045212 1950-01-01  258.828888
                     1951-01-01  258.898285
                     1952-01-01  258.709839
                     1953-01-01  258.461304
                     1954-01-01  259.764862
...                                     ...
40.041039 -74.045807 2096-01-01  287.735748
                     2097-01-01  288.222382
                     2098-01-01  287.898895
                     2099-01-01  288.221161
                     2100-01-01  288.576843

[126578166 rows x 1 columns]

In [75]:
dftgm = df11.reset_index()

In [76]:
dftgm

,lat,lon,time,tg_mean
0,66.623306,-89.045212,1950-01-01,258.828888
1,66.623306,-89.045212,1951-01-01,258.898285
2,66.623306,-89.045212,1952-01-01,258.709839
3,66.623306,-89.045212,1953-01-01,258.461304
4,66.623306,-89.045212,1954-01-01,259.764862
...,...,...,...,...
126578161,40.041039,-74.045807,2096-01-01,287.735748
126578162,40.041039,-74.045807,2097-01-01,288.222382
126578163,40.041039,-74.045807,2098-01-01,287.898895
126578164,40.041039,-74.045807,2099-01-01,288.221161


# Get the mean for 11 models

In [77]:
#Takes 48s
%time dftgm2 = dftgm.groupby(["lat", "lon", "time"]).mean()

CPU times: user 1min 9s, sys: 3min 24s, total: 4min 33s
Wall time: 1min 11s


In [78]:
dftgm2

tg_mean
lat       lon        time                  
40.041039 -89.045212 1950-01-01  284.512207
                     1951-01-01  284.301483
                     1952-01-01  284.201324
                     1953-01-01  284.058258
                     1954-01-01  284.223297
...                                     ...
66.623306 -61.296322 2096-01-01  266.947266
                     2097-01-01  266.583160
                     2098-01-01  267.383698
                     2099-01-01  267.084015
                     2100-01-01  266.929138

[11507106 rows x 1 columns]

# Transform temp to Celsius

In [79]:
dfC = dftgm2.copy()
dfC["tg_mean"] = dfC["tg_mean"] -273.15
    

# Define 3 periods of time: 

In [80]:
year_groups = {y:0 for y in range(1980,2011)}
year_groups.update({y:1 for y in range(2041,2071)})
year_groups.update({y:2 for y in range(2071,2101)})
#Get the mean of every period
dfC2 = dfC.reset_index()
dfp = dfC2.groupby([dfC2.time.dt.year.map(year_groups), "lat","lon"]).mean()

In [81]:
dfp.reset_index()

,time,lat,lon,tg_mean
0,0.0,40.041039,-89.045212,11.582137
1,0.0,40.041039,-88.961884,11.563147
2,0.0,40.041039,-88.878548,11.553229
3,0.0,40.041039,-88.795219,11.533276
4,0.0,40.041039,-88.711891,11.550668
...,...,...,...,...
228613,2.0,66.623306,-61.629639,-6.811722
228614,2.0,66.623306,-61.546310,-7.124964
228615,2.0,66.623306,-61.462978,-6.919032
228616,2.0,66.623306,-61.379650,-6.850576


# Period 2041-2071

In [102]:
df_h = dfp.query("time==0.0")
df_h.reset_index()

,time,lat,lon,tg_mean
0,0.0,40.041039,-89.045212,11.582137
1,0.0,40.041039,-88.961884,11.563147
2,0.0,40.041039,-88.878548,11.553229
3,0.0,40.041039,-88.795219,11.533276
4,0.0,40.041039,-88.711891,11.550668
...,...,...,...,...
76201,0.0,66.623306,-61.629639,-9.662173
76202,0.0,66.623306,-61.546310,-9.956597
76203,0.0,66.623306,-61.462978,-9.742899
76204,0.0,66.623306,-61.379650,-9.663411


In [103]:
df2050 = dfp.query("time==1.0")
df2050.reset_index()

,time,lat,lon,tg_mean
0,1.0,40.041039,-89.045212,13.636863
1,1.0,40.041039,-88.961884,13.619951
2,1.0,40.041039,-88.878548,13.611746
3,1.0,40.041039,-88.795219,13.591616
4,1.0,40.041039,-88.711891,13.611670
...,...,...,...,...
76201,1.0,66.623306,-61.629639,-7.389808
76202,1.0,66.623306,-61.546310,-7.699386
76203,1.0,66.623306,-61.462978,-7.491785
76204,1.0,66.623306,-61.379650,-7.421025


# Period 2071-2100

In [84]:
df2080 = dfp.query("time==2.0")
df2080.reset_index()

,time,lat,lon,tg_mean
0,2.0,40.041039,-89.045212,14.238521
1,2.0,40.041039,-88.961884,14.221740
2,2.0,40.041039,-88.878548,14.213413
3,2.0,40.041039,-88.795219,14.193439
4,2.0,40.041039,-88.711891,14.213731
...,...,...,...,...
76201,2.0,66.623306,-61.629639,-6.811722
76202,2.0,66.623306,-61.546310,-7.124964
76203,2.0,66.623306,-61.462978,-6.919032
76204,2.0,66.623306,-61.379650,-6.850576


## Import dataframe with polygons

In [85]:
dfpolyshape = from_geofeather('Grid-TerritoiresGuides.feather')

/home/mlopez/EXEC/anaconda3/envs/python37-2/lib/python3.8/site-packages/geofeather/core.py:85: UserWarning: Grid-TerritoiresGuides.feather.crs coordinate reference system file is missing. No crs will be set for this GeoDataFrame.
  warnings.warn(


In [44]:
dfpolyshape

,lat,lon,TER_GUIDE,geometry
0,52.957191,-67.712730,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
1,52.957191,-67.629402,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
2,52.957191,-67.546066,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
3,52.873859,-67.796059,6opqr,"POLYGON ((-67.75439 52.89329, -67.75439 52.832..."
4,52.873859,-67.712730,6opqr,"POLYGON ((-67.75439 52.83219, -67.75439 52.893..."
...,...,...,...,...
33193,45.041668,-72.208336,2c,"POLYGON ((-72.25000 45.00450, -72.25000 45.083..."
33194,45.041668,-72.125000,2c,"POLYGON ((-72.16666 45.00508, -72.16666 45.083..."
33195,45.041668,-72.041672,2c,"POLYGON ((-72.08334 45.00564, -72.08334 45.083..."
33196,45.041668,-71.958336,2c,"POLYGON ((-72.00000 45.00667, -72.00000 45.083..."


# Merge data with mask 

## Period 2041-2071

In [105]:
dftg2050TG = pd.merge(df2050, dfpolyshape, on=["lat","lon"])
dftg2050TG

,lat,lon,tg_mean,TER_GUIDE,geometry
0,44.957512,-74.629120,9.538018,1a,"POLYGON ((-74.62590 44.99918, -74.58746 44.999..."
1,44.957512,-74.545792,9.513206,1a,"POLYGON ((-74.58746 44.99876, -74.58746 44.999..."
2,44.957512,-74.462463,9.489746,1a,"POLYGON ((-74.50413 44.99784, -74.50413 44.999..."
3,44.957512,-74.379128,9.475520,1a,"POLYGON ((-74.42079 44.99530, -74.42079 44.999..."
4,44.957512,-74.295799,9.437252,1a,"POLYGON ((-74.33746 44.99215, -74.33746 44.999..."
...,...,...,...,...,...
16563,52.873859,-67.629402,-0.763553,6opqr,"POLYGON ((-67.67107 52.83219, -67.67107 52.915..."
16564,52.873859,-67.546066,-0.713376,6opqr,"POLYGON ((-67.58773 52.83219, -67.58773 52.904..."
16565,52.957191,-67.712730,-0.899334,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
16566,52.957191,-67.629402,-0.800437,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."


## Period 2071-2100

In [104]:
dftg2080TG = pd.merge(df2080, dfpolyshape, on=["lat","lon"])
dftg2080TG

,lat,lon,tg_mean,TER_GUIDE,geometry
0,44.957512,-74.629120,10.221003,1a,"POLYGON ((-74.62590 44.99918, -74.58746 44.999..."
1,44.957512,-74.545792,10.196143,1a,"POLYGON ((-74.58746 44.99876, -74.58746 44.999..."
2,44.957512,-74.462463,10.172487,1a,"POLYGON ((-74.50413 44.99784, -74.50413 44.999..."
3,44.957512,-74.379128,10.157830,1a,"POLYGON ((-74.42079 44.99530, -74.42079 44.999..."
4,44.957512,-74.295799,10.119123,1a,"POLYGON ((-74.33746 44.99215, -74.33746 44.999..."
...,...,...,...,...,...
16563,52.873859,-67.629402,-0.044033,6opqr,"POLYGON ((-67.67107 52.83219, -67.67107 52.915..."
16564,52.873859,-67.546066,0.005096,6opqr,"POLYGON ((-67.58773 52.83219, -67.58773 52.904..."
16565,52.957191,-67.712730,-0.180504,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
16566,52.957191,-67.629402,-0.082217,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."


# Create GeoJson for each territory

## Period 2041-2071

In [110]:
listTG2050 = []
for tg in dftg2050TG['TER_GUIDE'].unique().tolist():
    df = dftg2050TG[dftg2050TG.TER_GUIDE == tg]
    print (tg)
    listTG2050.append(df)
listTG2050

1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr


[            lat        lon   tg_mean TER_GUIDE  \
 0     44.957512 -74.629120  9.538018        1a   
 1     44.957512 -74.545792  9.513206        1a   
 2     44.957512 -74.462463  9.489746        1a   
 3     44.957512 -74.379128  9.475520        1a   
 4     44.957512 -74.295799  9.437252        1a   
 ...         ...        ...       ...       ...   
 1236  46.207458 -72.629204  7.885466        1a   
 1345  46.290791 -72.879189  7.743872        1a   
 1347  46.290791 -72.795860  7.787753        1a   
 1349  46.290791 -72.712532  7.804598        1a   
 1351  46.290791 -72.629204  7.801954        1a   
 
                                                geometry  
 0     POLYGON ((-74.62590 44.99918, -74.58746 44.999...  
 1     POLYGON ((-74.58746 44.99876, -74.58746 44.999...  
 2     POLYGON ((-74.50413 44.99784, -74.50413 44.999...  
 3     POLYGON ((-74.42079 44.99530, -74.42079 44.999...  
 4     POLYGON ((-74.33746 44.99215, -74.33746 44.999...  
 ...                            

In [111]:
for tg in listTG2050:
    geometry = tg["geometry"]
    crs = {'init': "epsg:4326"}
    gdf2050 = GeoDataFrame(tg, crs=crs, geometry=geometry)
    #print (gdf['TER_GUIDE'].iloc[0])
    gdf2050.to_file(gdf2050['TER_GUIDE'].iloc[0]+"_2050_tg_mean_annual_rcp45_50.json", driver="GeoJSON")

## Period 2041-2071

In [112]:
listTG2080 = []
for tg in dftg2080TG['TER_GUIDE'].unique().tolist():
    df = dftg2080TG[dftg2080TG.TER_GUIDE == tg]
    print (tg)
    listTG2080.append(df)
listTG2080

1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr


[            lat        lon    tg_mean TER_GUIDE  \
 0     44.957512 -74.629120  10.221003        1a   
 1     44.957512 -74.545792  10.196143        1a   
 2     44.957512 -74.462463  10.172487        1a   
 3     44.957512 -74.379128  10.157830        1a   
 4     44.957512 -74.295799  10.119123        1a   
 ...         ...        ...        ...       ...   
 1236  46.207458 -72.629204   8.595427        1a   
 1345  46.290791 -72.879189   8.456574        1a   
 1347  46.290791 -72.795860   8.500412        1a   
 1349  46.290791 -72.712532   8.517118        1a   
 1351  46.290791 -72.629204   8.513852        1a   
 
                                                geometry  
 0     POLYGON ((-74.62590 44.99918, -74.58746 44.999...  
 1     POLYGON ((-74.58746 44.99876, -74.58746 44.999...  
 2     POLYGON ((-74.50413 44.99784, -74.50413 44.999...  
 3     POLYGON ((-74.42079 44.99530, -74.42079 44.999...  
 4     POLYGON ((-74.33746 44.99215, -74.33746 44.999...  
 ...                

In [113]:
for tg in listTG2080:
    geometry = tg["geometry"]
    crs = {'init': "epsg:4326"}
    gdf2080 = GeoDataFrame(tg, crs=crs, geometry=geometry)
    #print (gdf['TER_GUIDE'].iloc[0])
    gdf2080.to_file(gdf2080['TER_GUIDE'].iloc[0]+"_2080_tg_mean_annual_rcp45_50.json", driver="GeoJSON")

# ---- Create GeoJson file for the complete shape----

## Period 2041-2071

In [114]:
geometry = dftg11TG["geometry"]
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftg2050TG, crs=crs, geometry=geometry)
gdf.to_file("TG_2050_tg_mean_annual_rcp45_50.json", driver="GeoJSON")

## Period 2071-2100

In [115]:
geometry = dftg11TG["geometry"]
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftg2080TG, crs=crs, geometry=geometry)
gdf.to_file("TG_2080_tg_mean_annual_rcp45_50.json", driver="GeoJSON")